In [1]:
import pandas as pd
from os import listdir
import numpy as np

In [2]:
def get_taxa_ranks(classification, taxa_rank="all"):
    mapping = {
        "domain": 'd',
        "phylum": 'p',
        "class": 'c',
        "order": 'o',
        "family": 'f',
        "genus": 'g',
        "species": 's',
    }
    
    tokens = classification.split(";")
    d = {}
    for tkn in tokens:
        rank = tkn[0]
        info = tkn[3:]
        
        d[rank] = info
    
    if taxa_rank == "all":
        return d
    elif taxa_rank == "domain":
        return d[mapping[taxa_rank]]
    elif taxa_rank == "phylum":
        return d[mapping[taxa_rank]]
    elif taxa_rank == "class":
        return d[mapping[taxa_rank]]
    elif taxa_rank == "order":
        return d[mapping[taxa_rank]]
    elif taxa_rank == "family":
        return d[mapping[taxa_rank]]
    elif taxa_rank == "genus":
        return d[mapping[taxa_rank]]
    else:
        assert taxa_rank == "species"
        return d[mapping[taxa_rank]]
        

In [3]:
samples = [
    "cc4_1", "cc4_2", "cc4_3",
    "cd4_1", "cd4_2", "cd4_3",
    "mc4_1", "mc4_2", "mc4_3",
    "md4_1", "md4_2", "md4_3",
    "fc4_1", "fc4_2", "fc4_3",
    "fd4_1", "fd4_2", "fd4_3",
]

In [4]:
# example = pd.read_csv(f"../results_metaspades/binning_based/gtdbtk/bacteria/cc4_2.csv")
# for i in example[example["Classification"].str.contains("Bacteroidota")].itertuples():
#     print(i.Classification)

In [5]:
taxa_ranks = ["domain", "phylum", "class", "order", "family", "genus", "species"]

In [6]:
DF_TAXA_RANKS = {
    "domain": {},
    "phylum": {},
    "class": {},
    "order": {},
    "family": {},
    "genus": {},
    "species": {},
}


for s in samples:
    flag = False
    if f"{s}.csv" in listdir("../results_metaspades/binning_based/gtdbtk/archaea"):
        flag = True
    
    bac_df = pd.read_csv(f"../results_metaspades/binning_based/gtdbtk/bacteria/{s}.csv")
    bac_df["bins"]   = bac_df.apply(lambda x: f'Bin{x["User Genome"].split(".")[1]}', axis=1)
    
    bac_df["domain"]  = bac_df.apply(lambda x: get_taxa_ranks(x["Classification"], "domain"), axis=1)
    bac_df["phylum"]  = bac_df.apply(lambda x: get_taxa_ranks(x["Classification"], "phylum"), axis=1)
    bac_df["class"]   = bac_df.apply(lambda x: get_taxa_ranks(x["Classification"], "class"), axis=1)
    bac_df["order"]   = bac_df.apply(lambda x: get_taxa_ranks(x["Classification"], "order"), axis=1)
    bac_df["family"]  = bac_df.apply(lambda x: get_taxa_ranks(x["Classification"], "family"), axis=1)
    bac_df["genus"]   = bac_df.apply(lambda x: get_taxa_ranks(x["Classification"], "genus"), axis=1)
    bac_df["species"] = bac_df.apply(lambda x: get_taxa_ranks(x["Classification"], "species"), axis=1)
    
    if flag:
        arc_df = pd.read_csv(f"../results_metaspades/binning_based/gtdbtk/archaea/{s}.csv")
        arc_df["bins"]   = arc_df.apply(lambda x: f'Bin{x["User Genome"].split(".")[1]}', axis=1)
        
        arc_df["domain"]  = arc_df.apply(lambda x: get_taxa_ranks(x["Classification"], "domain"), axis=1)
        arc_df["phylum"]  = arc_df.apply(lambda x: get_taxa_ranks(x["Classification"], "phylum"), axis=1)
        arc_df["class"]   = arc_df.apply(lambda x: get_taxa_ranks(x["Classification"], "class"), axis=1)
        arc_df["order"]   = arc_df.apply(lambda x: get_taxa_ranks(x["Classification"], "order"), axis=1)
        arc_df["family"]  = arc_df.apply(lambda x: get_taxa_ranks(x["Classification"], "family"), axis=1)
        arc_df["genus"]   = arc_df.apply(lambda x: get_taxa_ranks(x["Classification"], "genus"), axis=1)
        arc_df["species"] = arc_df.apply(lambda x: get_taxa_ranks(x["Classification"], "species"), axis=1)
    
        final = pd.concat([bac_df, arc_df], ignore_index=True, )
    
    else:
        final = bac_df
    
    for tr in taxa_ranks:
        DF_TAXA_RANKS[tr][s] = final.loc[:,[tr, "bins"]].groupby(by=tr).count().reset_index()


        
        
    



In [7]:
DF_TAXA_RANKS["phylum"]["cc4_2"]

,phylum,bins
0,Acidobacteriota,3
1,Actinobacteriota,3
2,Armatimonadota,1
3,Bacteroidota,19
4,Chloroflexota,1
5,Gemmatimonadota,3
6,Nitrospirota,1
7,Patescibacteria,1
8,Planctomycetota,1
9,Proteobacteria,7


In [8]:
DF_TAXA_RANKS["phylum"]["cc4_2"]

,phylum,bins
0,Acidobacteriota,3
1,Actinobacteriota,3
2,Armatimonadota,1
3,Bacteroidota,19
4,Chloroflexota,1
5,Gemmatimonadota,3
6,Nitrospirota,1
7,Patescibacteria,1
8,Planctomycetota,1
9,Proteobacteria,7


In [9]:
example = DF_TAXA_RANKS["order"]["fc4_1"].copy()
example

,order,bins
0,,1
1,Bdellovibrionales,1
2,Burkholderiales,2
3,Chitinophagales,8
4,Flavobacteriales,1
5,Micropepsales,1
6,Steroidobacterales,2


In [10]:
example = example.replace(r'^\s*$', "_unclassified_", regex=True)
example

,order,bins
0,_unclassified_,1
1,Bdellovibrionales,1
2,Burkholderiales,2
3,Chitinophagales,8
4,Flavobacteriales,1
5,Micropepsales,1
6,Steroidobacterales,2


In [11]:
def adapt_df_for_PCA(df, pivot):
    raw_newdf = df.T.reset_index()
    newdf = raw_newdf.replace(np.nan, 0, regex=True)

    new_header = newdf.iloc[0] #grab the first row for the header
    newdf = newdf[1:] #take the data less the header row
    newdf.columns = new_header #set the header row as the df header

    sample_id = {
        "cc4": 0,
        "cd4": 1,
        "mc4": 2,
        "md4": 3,
        "fc4": 4,
        "fd4": 5
    }

    newdf["sample"] = newdf.apply(lambda x: x[pivot].split("_")[0] , axis=1)
    newdf["sample_ids"] = newdf.apply(lambda x: sample_id[x["sample"]], axis=1)
    
    macro_sample_id = {
        "c": 0,
        "m": 1,
        "f": 2,
    }
    
    macro_sample_location = {
        "c": "Suna Canottieri",
        "m": "Teatro Maggiore",
        "f": "Fondo Toce",
    }
    
    newdf["macro_sample"] = newdf.apply(lambda x: x[pivot][0] , axis=1)
    newdf["macro_sample_ids"] = newdf.apply(lambda x: macro_sample_id[x["macro_sample"]], axis=1)
    newdf["macro_sample_location"] = newdf.apply(lambda x: macro_sample_location[x["macro_sample"]], axis=1)
    newdf["coast_deep_shapes"] = newdf.apply(lambda x: "coast" if x["sample"][1] == "c" else "deep", axis=1)
    
    return newdf

In [12]:
def merge_results_by_taxa(DF_TAXA_RANKS, taxa_level):
    assert taxa_level in ["domain", "phylum", "class", "order", "family", "genus", "species"]
    
    # remove empty classification
    for _, x in DF_TAXA_RANKS[taxa_level].items():
        x.replace(r'^\s*$', "unclassified_", regex=True, inplace=True)
    
    db_list = []
    for _, x in DF_TAXA_RANKS[taxa_level].items():
        db_list += list(x[taxa_level])
    
    unique_db_list = sorted(set(db_list))
    
    m = pd.DataFrame(unique_db_list, columns=[taxa_level])
    
    for s, x in DF_TAXA_RANKS[taxa_level].items():
        temp_sample_df = x.copy()
        temp_sample_df.rename(columns={"bins": s}, inplace=True)
        temp = pd.merge(m, temp_sample_df, how="left", on=taxa_level)
        m = temp.copy()
    
    newm = m.fillna(0)
    newm_adapted = adapt_df_for_PCA(newm, pivot=taxa_level)
    return newm_adapted
    

# Domain

In [13]:
domain_merged = merge_results_by_taxa(DF_TAXA_RANKS, taxa_level="domain")
domain_merged

,domain,Archaea,Bacteria,sample,sample_ids,macro_sample,macro_sample_ids,macro_sample_location,coast_deep_shapes
1,cc4_1,0.0,19.0,cc4,0,c,0,Suna Canottieri,coast
2,cc4_2,0.0,41.0,cc4,0,c,0,Suna Canottieri,coast
3,cc4_3,0.0,47.0,cc4,0,c,0,Suna Canottieri,coast
4,cd4_1,0.0,12.0,cd4,1,c,0,Suna Canottieri,deep
5,cd4_2,0.0,8.0,cd4,1,c,0,Suna Canottieri,deep
6,cd4_3,0.0,6.0,cd4,1,c,0,Suna Canottieri,deep
7,mc4_1,0.0,3.0,mc4,2,m,1,Teatro Maggiore,coast
8,mc4_2,0.0,4.0,mc4,2,m,1,Teatro Maggiore,coast
9,mc4_3,0.0,5.0,mc4,2,m,1,Teatro Maggiore,coast
10,md4_1,0.0,15.0,md4,3,m,1,Teatro Maggiore,deep


# Phylum

In [14]:
phylum_merged = merge_results_by_taxa(DF_TAXA_RANKS, taxa_level="phylum")
phylum_merged.to_csv("./binning-based_gtdbtk/phylum.tsv", sep="\t", header=True, index=False)
phylum_merged

,phylum,Acidobacteriota,Actinobacteriota,Armatimonadota,Bacteroidota,Bdellovibrionota,Chloroflexota,Desulfobacterota,Gemmatimonadota,Myxococcota,...,Proteobacteria,Thermoplasmatota,Thermoproteota,Verrucomicrobiota,sample,sample_ids,macro_sample,macro_sample_ids,macro_sample_location,coast_deep_shapes
1,cc4_1,1.0,1.0,1.0,8.0,0.0,0.0,0.0,2.0,0.0,...,5.0,0.0,0.0,0.0,cc4,0,c,0,Suna Canottieri,coast
2,cc4_2,3.0,3.0,1.0,19.0,0.0,1.0,0.0,3.0,0.0,...,7.0,0.0,0.0,1.0,cc4,0,c,0,Suna Canottieri,coast
3,cc4_3,5.0,4.0,1.0,19.0,0.0,1.0,0.0,3.0,1.0,...,8.0,0.0,0.0,1.0,cc4,0,c,0,Suna Canottieri,coast
4,cd4_1,1.0,2.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
5,cd4_2,1.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
6,cd4_3,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
7,mc4_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
8,mc4_2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
9,mc4_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
10,md4_1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,9.0,0.0,0.0,0.0,md4,3,m,1,Teatro Maggiore,deep


# Class

In [15]:
class_merged = merge_results_by_taxa(DF_TAXA_RANKS, taxa_level="class")
class_merged.to_csv("./binning-based_gtdbtk/class.tsv", sep="\t", header=True, index=False)
class_merged

,class,Acidimicrobiia,Actinomycetia,Alphaproteobacteria,Anaerolineae,Armatimonadia,Bacteroidia,Bdellovibrionia,Blastocatellia,Desulfobacteria,...,UBA9160,UBA9217,Verrucomicrobiae,Vicinamibacteria,sample,sample_ids,macro_sample,macro_sample_ids,macro_sample_location,coast_deep_shapes
1,cc4_1,1.0,0.0,1.0,0.0,1.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,cc4,0,c,0,Suna Canottieri,coast
2,cc4_2,3.0,0.0,3.0,1.0,1.0,18.0,0.0,2.0,0.0,...,0.0,0.0,1.0,1.0,cc4,0,c,0,Suna Canottieri,coast
3,cc4_3,3.0,0.0,4.0,1.0,1.0,18.0,0.0,4.0,0.0,...,1.0,0.0,1.0,1.0,cc4,0,c,0,Suna Canottieri,coast
4,cd4_1,1.0,1.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
5,cd4_2,1.0,1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
6,cd4_3,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
7,mc4_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
8,mc4_2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
9,mc4_3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
10,md4_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,md4,3,m,1,Teatro Maggiore,deep


# Order

In [16]:
order_merged = merge_results_by_taxa(DF_TAXA_RANKS, taxa_level="order")
order_merged.to_csv("./binning-based_gtdbtk/order.tsv", sep="\t", header=True, index=False)
order_merged

,order,AKYH767-A,Acidiferrobacterales,Acidimicrobiales,Actinomycetales,Armatimonadales,BD1-5,Bacteroidales,Bdellovibrionales,Beggiatoales,...,UBA9983_A,Verrucomicrobiales,Vicinamibacterales,unclassified_,sample,sample_ids,macro_sample,macro_sample_ids,macro_sample_location,coast_deep_shapes
1,cc4_1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,cc4,0,c,0,Suna Canottieri,coast
2,cc4_2,0.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,cc4,0,c,0,Suna Canottieri,coast
3,cc4_3,0.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,cc4,0,c,0,Suna Canottieri,coast
4,cd4_1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
5,cd4_2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
6,cd4_3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
7,mc4_1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
8,mc4_2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
9,mc4_3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
10,md4_1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,md4,3,m,1,Teatro Maggiore,deep


# Family

In [17]:
family_merged = merge_results_by_taxa(DF_TAXA_RANKS, taxa_level="family")
family_merged.to_csv("./binning-based_gtdbtk/family.tsv", sep="\t", header=True, index=False)
family_merged

,family,2013-40CM-41-45,Aestuariivirgaceae,Armatimonadaceae,Bdellovibrionaceae,Beggiatoaceae,Burkholderiaceae,Chitinophagaceae,Chromatiaceae,Cyclobacteriaceae,...,VadinHA17,Verrucomicrobiaceae,WLNW01,unclassified_,sample,sample_ids,macro_sample,macro_sample_ids,macro_sample_location,coast_deep_shapes
1,cc4_1,0.0,1.0,1.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,1.0,cc4,0,c,0,Suna Canottieri,coast
2,cc4_2,0.0,2.0,1.0,0.0,0.0,3.0,11.0,0.0,1.0,...,0.0,1.0,1.0,2.0,cc4,0,c,0,Suna Canottieri,coast
3,cc4_3,0.0,3.0,1.0,0.0,0.0,3.0,9.0,0.0,1.0,...,0.0,1.0,1.0,2.0,cc4,0,c,0,Suna Canottieri,coast
4,cd4_1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,1.0,cd4,1,c,0,Suna Canottieri,deep
5,cd4_2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
6,cd4_3,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,cd4,1,c,0,Suna Canottieri,deep
7,mc4_1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
8,mc4_2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
9,mc4_3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,mc4,2,m,1,Teatro Maggiore,coast
10,md4_1,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,md4,3,m,1,Teatro Maggiore,deep


# Genus

In [18]:
genus_merged = merge_results_by_taxa(DF_TAXA_RANKS, taxa_level="genus")
genus_merged.to_csv("./binning-based_gtdbtk/genus.tsv", sep="\t", header=True, index=False)
genus_merged

,genus,Aestuariivirga,Archangium_A,Armatimonas,Azonexus,BJ22,CADEED01,CADEEN01,CADEFD01,CAINEC01,...,VFJN01,WLNW01,ZC4RG30,unclassified_,sample,sample_ids,macro_sample,macro_sample_ids,macro_sample_location,coast_deep_shapes
1,cc4_1,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,2.0,cc4,0,c,0,Suna Canottieri,coast
2,cc4_2,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,6.0,cc4,0,c,0,Suna Canottieri,coast
3,cc4_3,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,10.0,cc4,0,c,0,Suna Canottieri,coast
4,cd4_1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,cd4,1,c,0,Suna Canottieri,deep
5,cd4_2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,cd4,1,c,0,Suna Canottieri,deep
6,cd4_3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,cd4,1,c,0,Suna Canottieri,deep
7,mc4_1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,mc4,2,m,1,Teatro Maggiore,coast
8,mc4_2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,mc4,2,m,1,Teatro Maggiore,coast
9,mc4_3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,mc4,2,m,1,Teatro Maggiore,coast
10,md4_1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,7.0,md4,3,m,1,Teatro Maggiore,deep


# Species

In [19]:
species_merged = merge_results_by_taxa(DF_TAXA_RANKS, taxa_level="species")
species_merged.to_csv("./binning-based_gtdbtk/species.tsv", sep="\t", header=True, index=False)
species_merged

,species,Nitrospira_F sp001464735,Nitrospira_F sp002083565,Sphingorhabdus_B lacus,UBA3362 sp013140975,unclassified_,sample,sample_ids,macro_sample,macro_sample_ids,macro_sample_location,coast_deep_shapes
1,cc4_1,0.0,1.0,0.0,0.0,18.0,cc4,0,c,0,Suna Canottieri,coast
2,cc4_2,1.0,0.0,0.0,1.0,39.0,cc4,0,c,0,Suna Canottieri,coast
3,cc4_3,0.0,1.0,0.0,1.0,45.0,cc4,0,c,0,Suna Canottieri,coast
4,cd4_1,1.0,1.0,0.0,1.0,9.0,cd4,1,c,0,Suna Canottieri,deep
5,cd4_2,0.0,0.0,0.0,0.0,8.0,cd4,1,c,0,Suna Canottieri,deep
6,cd4_3,0.0,0.0,0.0,0.0,6.0,cd4,1,c,0,Suna Canottieri,deep
7,mc4_1,0.0,0.0,0.0,0.0,3.0,mc4,2,m,1,Teatro Maggiore,coast
8,mc4_2,0.0,0.0,0.0,0.0,4.0,mc4,2,m,1,Teatro Maggiore,coast
9,mc4_3,0.0,0.0,0.0,0.0,5.0,mc4,2,m,1,Teatro Maggiore,coast
10,md4_1,0.0,0.0,0.0,0.0,15.0,md4,3,m,1,Teatro Maggiore,deep
